In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os

In [2]:
def load_data(input_path):
    data = []
    labels = []
    for file in os.listdir(input_path):
        if file.startswith("x"):
            data.append(pd.read_csv(os.path.join(input_path, file), 
                                    index_col=False, 
                                    usecols=['speed',
                                    'flow_per_lane',
                                    'before_on_ramp',
                                    'after_on_ramp',
                                    'before_off_ramp',
                                    'after_off_ramp']).to_numpy())
        if file.startswith("y"):
            labels.append(pd.read_csv(os.path.join(input_path, file), index_col=False, header=None).to_numpy().transpose()[0])
            #print(labels)
            #break
    return np.array(data), np.array(labels)
    #return tf.data.Dataset.from_tensor_slices((np.array(data), np.array(labels)))

In [3]:
features, labels = load_data("C:\Aimsun Projects\Calibration Using Neural Networks\dataset")
# shuffling and splitting data
indices = np.random.permutation(features.shape[0])
training_idx, test_idx = indices[:450], indices[450:]
train_features, test_features = features[training_idx,:], features[test_idx,:]
train_labels, test_labels = labels[training_idx,:], labels[test_idx,:]
print("Training set: {}, {}".format(train_features.shape, train_labels.shape))
print("Test set: {}, {}".format(test_features.shape, test_labels.shape))
#features[training_idx[0]][:5], train_features[0][:5]
#labels[training_idx[0]][:5], train_labels[0][:5]

Training set: (450, 162, 6), (450, 9)
Test set: (92, 162, 6), (92, 9)


In [35]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(162,6)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(9)
])
#model.summary()
model.compile(loss='mse', optimizer=tf.train.RMSPropOptimizer(0.001), metrics=['mae'])
model.fit(train_features, train_labels, epochs=500, validation_split=0.2)

In [41]:
[loss, mae] = model.evaluate(test_features, test_labels)
print("Testing set Mean Abs Error: {}".format(mae))

92/92 [==============================] - 0s 54us/sample - loss: 8593.6808 - mean_absolute_error: 48.4932
Testing set Mean Abs Error: 48.493160247802734


In [64]:
# Normalizing speed and flow (normal distribution)
mean_features = train_features[:,:,:2].mean(axis=0).mean(axis=0)
std_features = train_features[:,:,:2].std(axis=0).mean(axis=0)
print("mean={}, std={}".format(mean_features, std_features))
train_features_norm = np.zeros_like(train_features)
test_features_norm = np.zeros_like(test_features)
train_features_norm[:,:,:2] = (train_features[:,:,:2] - mean_features) / std_features
train_features_norm[:,:,2:] = train_features[:,:,2:]
test_features_norm[:,:,:2] = (test_features[:,:,:2] - mean_features) / std_features
test_features_norm[:,:,2:] = test_features[:,:,2:]

mean=[  80.36278264 1032.39972565], std=[ 17.11294444 190.56003764]


In [74]:
model_norm = keras.Sequential([
    keras.layers.Flatten(input_shape=(162,6)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(9)
])
model_norm.compile(loss='mse', optimizer=tf.train.RMSPropOptimizer(0.001), metrics=['mae'])
model_norm.fit(train_features_norm, train_labels, epochs=500, validation_split=0.2)

In [77]:
[loss, mae] = model_norm.evaluate(test_features_norm, test_labels)
print("Testing set Mean Abs Error: {}".format(mae))

92/92 [==============================] - 0s 76us/sample - loss: 1785.1408 - mean_absolute_error: 23.6431
Testing set Mean Abs Error: 23.643112182617188


In [80]:
model_norm.predict(test_features_norm[:1])

array([[4.5853138e+00, 6.3562906e-01, 3.1949904e+00, 4.8319273e+00,
        5.4238220e+01, 4.8456871e+01, 1.7729977e+02, 6.8305389e+02,
        6.8358759e+02]], dtype=float32)

In [87]:
test_labels[0]

array([3.0e-01, 7.0e-01, 1.1e+00, 4.0e+00, 7.6e+01, 6.6e+01, 8.5e+01,
       6.5e+02, 6.5e+02])

In [82]:
# Normalizing outputs
mean_labels = train_labels.mean(axis=0)
std_labels = train_labels.std(axis=0)
print("mean={}, std={}".format(mean_labels, std_labels))
train_labels_norm = (train_labels - mean_labels) / std_labels
test_labels_norm = (test_labels - mean_labels) / std_labels

mean=[8.15777778e-01 8.22222222e-03 2.45888889e+00 2.99666667e+00
 4.97622222e+01 5.10377778e+01 1.55133333e+02 5.88755556e+02
 5.88755556e+02], std=[  0.44301487   0.61971244   0.89244912   1.16318051  28.86256929
  29.57815852  85.58740821 242.55997247 242.55997247]


In [83]:
model_norm_inout = keras.Sequential([
    keras.layers.Flatten(input_shape=(162,6)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(9, activation=tf.nn.sigmoid)
])
model_norm_inout.compile(loss='mse', optimizer=tf.train.RMSPropOptimizer(0.001), metrics=['mae'])
model_norm_inout.fit(train_features_norm, train_labels_norm, epochs=500, validation_split=0.2)

In [88]:
[loss, mae] = model_norm_inout.evaluate(test_features_norm, test_labels_norm)
print("Testing set Mean Abs Error: {}".format(mae))

92/92 [==============================] - 0s 76us/sample - loss: 0.8676 - mean_absolute_error: 0.7421
Testing set Mean Abs Error: 0.7421111464500427


In [91]:
model_norm.predict(test_features_norm[:1]) * std + mean

array([[-3.35068161e-03,  5.80067930e-01,  5.06294719e-01,
         5.27618685e+00,  6.07226956e+01,  9.47037982e+01,
         2.43443475e+02,  9.82865314e+02,  1.00666174e+03]])

In [92]:
test_labels[0]

array([3.0e-01, 7.0e-01, 1.1e+00, 4.0e+00, 7.6e+01, 6.6e+01, 8.5e+01,
       6.5e+02, 6.5e+02])

In [4]:
# Min/Max Normalizing
minimum_features = train_features[:,:,:2].min(axis=0).min(axis=0)
maximum_features = train_features[:,:,:2].max(axis=0).max(axis=0)
print("min={}, max={}".format(minimum_features, maximum_features))
train_features_minmax = train_features
test_features_minmax = test_features
train_features_minmax[:,:,:2] = (train_features[:,:,:2] - minimum_features) / (maximum_features - minimum_features)
test_features_minmax[:,:,:2] = (test_features[:,:,:2] - minimum_features) / (maximum_features - minimum_features)

minimum_labels = train_labels.min(axis=0)
maximum_labels = train_labels.max(axis=0)
print("min={}, max={}".format(minimum_labels, maximum_labels))
train_labels_minmax = (train_labels - minimum_labels) / (maximum_labels - minimum_labels)
test_labels_minmax = (test_labels - minimum_labels) / (maximum_labels - minimum_labels)

min=[  4.92314071 190.        ], max=[ 106.11641829 3090.        ]
min=[ 1.e-01 -1.e+00  1.e+00  1.e+00  0.e+00  0.e+00  0.e+00  2.e+02  2.e+02], max=[   1.5    1.     4.     5.   100.   100.   300.  1000.  1000. ]


In [5]:
model_norm_minmax = keras.Sequential([
    keras.layers.Flatten(input_shape=(162,6)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(9, activation=tf.nn.sigmoid)
])
model_norm_minmax.compile(loss='mse', 
                          optimizer=tf.train.RMSPropOptimizer(0.001), 
                          metrics=['mae'])
model_norm_minmax.fit(train_features_minmax, train_labels_minmax, epochs=500, validation_split=0.2)

W0726 16:49:28.516292  3652 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0726 16:49:28.668328  3652 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\training\rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 360 samples, validate on 90 samples
Epoch 1/500
360/360 [==============================] - 0s 314us/sample - loss: 0.0945 - mean_absolute_error: 0.2638 - val_loss: 0.1034 - val_mean_absolute_error: 0.2763
Epoch 2/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0938 - mean_absolute_error: 0.2632 - val_loss: 0.1022 - val_mean_absolute_error: 0.2749
Epoch 3/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0927 - mean_absolute_error: 0.2622 - val_loss: 0.1003 - val_mean_absolute_error: 0.2729
Epoch 4/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0911 - mean_absolute_error: 0.2610 - val_loss: 0.0977 - val_mean_absolute_error: 0.2700
Epoch 5/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0892 - mean_absolute_error: 0.2592 - val_loss: 0.0952 - val_mean_absolute_error: 0.2669
Epoch 6/500
360/360 [==============================] - 0s 94us/sample - loss: 0.0878 - mean_absolute_error: 0.2575

360/360 [==============================] - 0s 75us/sample - loss: 0.0594 - mean_absolute_error: 0.1993 - val_loss: 0.0654 - val_mean_absolute_error: 0.2080
Epoch 50/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0586 - mean_absolute_error: 0.1944 - val_loss: 0.0871 - val_mean_absolute_error: 0.2417
Epoch 51/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0593 - mean_absolute_error: 0.1973 - val_loss: 0.0650 - val_mean_absolute_error: 0.2059
Epoch 52/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0598 - mean_absolute_error: 0.2000 - val_loss: 0.0693 - val_mean_absolute_error: 0.2136
Epoch 53/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0593 - mean_absolute_error: 0.1991 - val_loss: 0.0692 - val_mean_absolute_error: 0.2095
Epoch 54/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0590 - mean_absolute_error: 0.1974 - val_loss: 0.0671 - val_mean_absolute_error: 0.2145

360/360 [==============================] - 0s 67us/sample - loss: 0.0503 - mean_absolute_error: 0.1788 - val_loss: 0.0566 - val_mean_absolute_error: 0.1908
Epoch 98/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0499 - mean_absolute_error: 0.1777 - val_loss: 0.0645 - val_mean_absolute_error: 0.2029
Epoch 99/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0499 - mean_absolute_error: 0.1771 - val_loss: 0.0584 - val_mean_absolute_error: 0.1921
Epoch 100/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0508 - mean_absolute_error: 0.1795 - val_loss: 0.0586 - val_mean_absolute_error: 0.1901
Epoch 101/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0494 - mean_absolute_error: 0.1744 - val_loss: 0.0638 - val_mean_absolute_error: 0.2022
Epoch 102/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0494 - mean_absolute_error: 0.1763 - val_loss: 0.0616 - val_mean_absolute_error: 0.2

360/360 [==============================] - 0s 61us/sample - loss: 0.0478 - mean_absolute_error: 0.1727 - val_loss: 0.0578 - val_mean_absolute_error: 0.1899
Epoch 146/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0449 - mean_absolute_error: 0.1652 - val_loss: 0.0561 - val_mean_absolute_error: 0.1872
Epoch 147/500
360/360 [==============================] - 0s 64us/sample - loss: 0.0459 - mean_absolute_error: 0.1690 - val_loss: 0.0502 - val_mean_absolute_error: 0.1755
Epoch 148/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0454 - mean_absolute_error: 0.1668 - val_loss: 0.0628 - val_mean_absolute_error: 0.1983
Epoch 149/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0463 - mean_absolute_error: 0.1688 - val_loss: 0.0614 - val_mean_absolute_error: 0.1997
Epoch 150/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0453 - mean_absolute_error: 0.1650 - val_loss: 0.0603 - val_mean_absolute_error: 0

...

360/360 [==============================] - 0s 72us/sample - loss: 0.0382 - mean_absolute_error: 0.1475 - val_loss: 0.0567 - val_mean_absolute_error: 0.1861
Epoch 290/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0370 - mean_absolute_error: 0.1440 - val_loss: 0.0584 - val_mean_absolute_error: 0.1859
Epoch 291/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0379 - mean_absolute_error: 0.1467 - val_loss: 0.0546 - val_mean_absolute_error: 0.1776
Epoch 292/500
360/360 [==============================] - 0s 61us/sample - loss: 0.0385 - mean_absolute_error: 0.1490 - val_loss: 0.0547 - val_mean_absolute_error: 0.1772
Epoch 293/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0353 - mean_absolute_error: 0.1401 - val_loss: 0.0568 - val_mean_absolute_error: 0.1808
Epoch 294/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0391 - mean_absolute_error: 0.1507 - val_loss: 0.0640 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 78us/sample - loss: 0.0351 - mean_absolute_error: 0.1403 - val_loss: 0.0610 - val_mean_absolute_error: 0.1894
Epoch 338/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0366 - mean_absolute_error: 0.1444 - val_loss: 0.0525 - val_mean_absolute_error: 0.1714
Epoch 339/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0339 - mean_absolute_error: 0.1370 - val_loss: 0.0531 - val_mean_absolute_error: 0.1760
Epoch 340/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0367 - mean_absolute_error: 0.1446 - val_loss: 0.0522 - val_mean_absolute_error: 0.1715
Epoch 341/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0345 - mean_absolute_error: 0.1396 - val_loss: 0.0639 - val_mean_absolute_error: 0.1982
Epoch 342/500
360/360 [==============================] - ETA: 0s - loss: 0.0442 - mean_absolute_error: 0.161 - 0s 67us/sample - loss: 0.0351 - mean_absolute_error: 

Epoch 385/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0351 - mean_absolute_error: 0.1395 - val_loss: 0.0552 - val_mean_absolute_error: 0.1728
Epoch 386/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0332 - mean_absolute_error: 0.1342 - val_loss: 0.0600 - val_mean_absolute_error: 0.1840
Epoch 387/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0336 - mean_absolute_error: 0.1370 - val_loss: 0.0606 - val_mean_absolute_error: 0.1845
Epoch 388/500
360/360 [==============================] - 0s 103us/sample - loss: 0.0327 - mean_absolute_error: 0.1319 - val_loss: 0.0616 - val_mean_absolute_error: 0.1900
Epoch 389/500
360/360 [==============================] - 0s 92us/sample - loss: 0.0336 - mean_absolute_error: 0.1369 - val_loss: 0.0587 - val_mean_absolute_error: 0.1812
Epoch 390/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0337 - mean_absolute_error: 0.1367 - val_loss: 0.0539 - val_mean_ab

360/360 [==============================] - 0s 78us/sample - loss: 0.0302 - mean_absolute_error: 0.1266 - val_loss: 0.0540 - val_mean_absolute_error: 0.1733
Epoch 434/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0334 - mean_absolute_error: 0.1342 - val_loss: 0.0575 - val_mean_absolute_error: 0.1792
Epoch 435/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0316 - mean_absolute_error: 0.1318 - val_loss: 0.0586 - val_mean_absolute_error: 0.1846
Epoch 436/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0325 - mean_absolute_error: 0.1325 - val_loss: 0.0532 - val_mean_absolute_error: 0.1700
Epoch 437/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0304 - mean_absolute_error: 0.1285 - val_loss: 0.0567 - val_mean_absolute_error: 0.1820
Epoch 438/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0309 - mean_absolute_error: 0.1290 - val_loss: 0.0597 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 72us/sample - loss: 0.0292 - mean_absolute_error: 0.1268 - val_loss: 0.0569 - val_mean_absolute_error: 0.1794
Epoch 482/500
360/360 [==============================] - 0s 61us/sample - loss: 0.0300 - mean_absolute_error: 0.1268 - val_loss: 0.0587 - val_mean_absolute_error: 0.1775
Epoch 483/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0299 - mean_absolute_error: 0.1268 - val_loss: 0.0573 - val_mean_absolute_error: 0.1819
Epoch 484/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0309 - mean_absolute_error: 0.1307 - val_loss: 0.0536 - val_mean_absolute_error: 0.1703
Epoch 485/500
360/360 [==============================] - 0s 64us/sample - loss: 0.0292 - mean_absolute_error: 0.1235 - val_loss: 0.0584 - val_mean_absolute_error: 0.1844
Epoch 486/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0300 - mean_absolute_error: 0.1266 - val_loss: 0.0594 - val_mean_absolute_error: 0

In [6]:
[loss, mae] = model_norm_minmax.evaluate(test_features_minmax, test_labels_minmax)
print("Testing set Mean Abs Error: {}".format(mae))

92/92 [==============================] - 0s 54us/sample - loss: 0.0562 - mean_absolute_error: 0.1748
Testing set Mean Abs Error: 0.17480653524398804


In [8]:
model_norm_minmax.predict(test_features_minmax).min(axis=0)

array([0.13739717, 0.07625419, 0.0629856 , 0.01202238, 0.20691523,
       0.18444782, 0.21345642, 0.04679623, 0.04720408], dtype=float32)

In [9]:
model_norm_minmax.predict(test_features_minmax).max(axis=0)

array([0.94233966, 0.95655024, 0.942437  , 0.8517747 , 0.90079486,
       0.92190486, 0.8188884 , 0.98566854, 0.9816543 ], dtype=float32)

In [10]:
test_labels_minmax.min(axis=0)

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
test_labels_minmax.max(axis=0)

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.98333333, 0.9875    , 0.9875    ])

In [12]:
model_norm_minmax.predict(test_features_minmax[:1]) * (maximum_labels - minimum_labels) + minimum_labels

array([[ 3.02129292e-01, -3.20344448e-01,  2.38692066e+00,
         2.78397369e+00,  4.99191552e+01,  4.95411813e+01,
         1.84131896e+02,  3.59860349e+02,  3.61088133e+02]])

In [13]:
test_labels_minmax[0] * (maximum_labels - minimum_labels) + minimum_labels

array([ 2.0e-01, -7.0e-01,  2.7e+00,  2.6e+00,  3.8e+01,  8.3e+01,
        2.1e+02,  3.0e+02,  3.0e+02])

In [25]:
sample1 = np.zeros_like(features[:1])
sample1[:1,:,:2] = (features[:1,:,:2] - minimum_features) / (maximum_features - minimum_features)
sample1[:1,:,2:] = features[:1,:,2:]
model_norm_minmax.predict(sample1) * (maximum_labels - minimum_labels) + minimum_labels

array([[ 3.14308560e-01, -3.00958633e-01,  2.36795333e+00,
         3.09285426e+00,  5.47510862e+01,  4.78829384e+01,
         1.75575435e+02,  4.09558368e+02,  4.06648755e+02]])

In [16]:
labels[0]

array([  0.5,   0.3,   2.1,   2.9,  36. ,   4. , 210. , 220. , 220. ])

In [19]:
model_norm_minmax.save("C:/Aimsun Projects/Calibration Using Neural Networks/normalized_minmax_model.h5")
np.save("C:/Aimsun Projects/Calibration Using Neural Networks/normalized_minmax_model_min_x.npy", minimum_features)
np.save("C:/Aimsun Projects/Calibration Using Neural Networks/normalized_minmax_model_max_x.npy", maximum_features)
np.save("C:/Aimsun Projects/Calibration Using Neural Networks/normalized_minmax_model_min_y.npy", minimum_labels)
np.save("C:/Aimsun Projects/Calibration Using Neural Networks/normalized_minmax_model_max_y.npy", maximum_labels)

W0726 16:58:01.863182  3652 hdf5_format.py:110] TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


In [23]:
model_norm_minmax2 = keras.Sequential([
    keras.layers.Flatten(input_shape=(162,6)),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(9, activation=tf.nn.sigmoid)
])
model_norm_minmax2.compile(loss='mse', 
                          optimizer=tf.train.RMSPropOptimizer(0.001), 
                          metrics=['mae'])
model_norm_minmax2.fit(train_features_minmax, train_labels_minmax, epochs=500, validation_split=0.2)

Train on 360 samples, validate on 90 samples
Epoch 1/500
360/360 [==============================] - 0s 350us/sample - loss: 0.1153 - mean_absolute_error: 0.2872 - val_loss: 0.1099 - val_mean_absolute_error: 0.2788
Epoch 2/500
360/360 [==============================] - 0s 86us/sample - loss: 0.1126 - mean_absolute_error: 0.2847 - val_loss: 0.1071 - val_mean_absolute_error: 0.2764
Epoch 3/500
360/360 [==============================] - 0s 91us/sample - loss: 0.1079 - mean_absolute_error: 0.2803 - val_loss: 0.1027 - val_mean_absolute_error: 0.2727
Epoch 4/500
360/360 [==============================] - 0s 103us/sample - loss: 0.1012 - mean_absolute_error: 0.2742 - val_loss: 0.0982 - val_mean_absolute_error: 0.2697
Epoch 5/500
360/360 [==============================] - 0s 106us/sample - loss: 0.0947 - mean_absolute_error: 0.2675 - val_loss: 0.0954 - val_mean_absolute_error: 0.2682
Epoch 6/500
360/360 [==============================] - 0s 103us/sample - loss: 0.0905 - mean_absolute_error: 0.2

360/360 [==============================] - 0s 81us/sample - loss: 0.0601 - mean_absolute_error: 0.2005 - val_loss: 0.0676 - val_mean_absolute_error: 0.2170
Epoch 50/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0607 - mean_absolute_error: 0.2010 - val_loss: 0.0661 - val_mean_absolute_error: 0.2121
Epoch 51/500
360/360 [==============================] - 0s 94us/sample - loss: 0.0593 - mean_absolute_error: 0.1996 - val_loss: 0.0771 - val_mean_absolute_error: 0.2244
Epoch 52/500
360/360 [==============================] - 0s 94us/sample - loss: 0.0623 - mean_absolute_error: 0.2053 - val_loss: 0.0721 - val_mean_absolute_error: 0.2168
Epoch 53/500
360/360 [==============================] - 0s 125us/sample - loss: 0.0588 - mean_absolute_error: 0.1976 - val_loss: 0.0665 - val_mean_absolute_error: 0.2077
Epoch 54/500
360/360 [==============================] - 0s 114us/sample - loss: 0.0592 - mean_absolute_error: 0.1991 - val_loss: 0.0631 - val_mean_absolute_error: 0.20

360/360 [==============================] - 0s 81us/sample - loss: 0.0505 - mean_absolute_error: 0.1789 - val_loss: 0.0647 - val_mean_absolute_error: 0.2022
Epoch 98/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0527 - mean_absolute_error: 0.1825 - val_loss: 0.0567 - val_mean_absolute_error: 0.1887
Epoch 99/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0525 - mean_absolute_error: 0.1832 - val_loss: 0.0607 - val_mean_absolute_error: 0.1946
Epoch 100/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0534 - mean_absolute_error: 0.1855 - val_loss: 0.0562 - val_mean_absolute_error: 0.1934
Epoch 101/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0517 - mean_absolute_error: 0.1801 - val_loss: 0.0579 - val_mean_absolute_error: 0.1881
Epoch 102/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0522 - mean_absolute_error: 0.1823 - val_loss: 0.0566 - val_mean_absolute_error: 0.1

360/360 [==============================] - 0s 89us/sample - loss: 0.0492 - mean_absolute_error: 0.1749 - val_loss: 0.0647 - val_mean_absolute_error: 0.1997
Epoch 146/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0476 - mean_absolute_error: 0.1696 - val_loss: 0.0589 - val_mean_absolute_error: 0.1859
Epoch 147/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0482 - mean_absolute_error: 0.1722 - val_loss: 0.0569 - val_mean_absolute_error: 0.1924
Epoch 148/500
360/360 [==============================] - 0s 98us/sample - loss: 0.0480 - mean_absolute_error: 0.1724 - val_loss: 0.0540 - val_mean_absolute_error: 0.1815
Epoch 149/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0491 - mean_absolute_error: 0.1758 - val_loss: 0.0544 - val_mean_absolute_error: 0.1865
Epoch 150/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0483 - mean_absolute_error: 0.1737 - val_loss: 0.0551 - val_mean_absolute_error: 0

Epoch 193/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0455 - mean_absolute_error: 0.1682 - val_loss: 0.0600 - val_mean_absolute_error: 0.1908
Epoch 194/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0459 - mean_absolute_error: 0.1669 - val_loss: 0.0655 - val_mean_absolute_error: 0.2027
Epoch 195/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0465 - mean_absolute_error: 0.1678 - val_loss: 0.0599 - val_mean_absolute_error: 0.1925
Epoch 196/500
360/360 [==============================] - 0s 100us/sample - loss: 0.0441 - mean_absolute_error: 0.1618 - val_loss: 0.0557 - val_mean_absolute_error: 0.1810
Epoch 197/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0484 - mean_absolute_error: 0.1734 - val_loss: 0.0663 - val_mean_absolute_error: 0.2026
Epoch 198/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0455 - mean_absolute_error: 0.1661 - val_loss: 0.0518 - val_mean_ab

...

360/360 [==============================] - 0s 89us/sample - loss: 0.0426 - mean_absolute_error: 0.1574 - val_loss: 0.0588 - val_mean_absolute_error: 0.1834
Epoch 290/500
360/360 [==============================] - 0s 97us/sample - loss: 0.0425 - mean_absolute_error: 0.1582 - val_loss: 0.0552 - val_mean_absolute_error: 0.1792
Epoch 291/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0433 - mean_absolute_error: 0.1597 - val_loss: 0.0563 - val_mean_absolute_error: 0.1840
Epoch 292/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0408 - mean_absolute_error: 0.1535 - val_loss: 0.0526 - val_mean_absolute_error: 0.1766
Epoch 293/500
360/360 [==============================] - 0s 92us/sample - loss: 0.0429 - mean_absolute_error: 0.1594 - val_loss: 0.0589 - val_mean_absolute_error: 0.1871
Epoch 294/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0419 - mean_absolute_error: 0.1559 - val_loss: 0.0578 - val_mean_absolute_error: 0

Epoch 337/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0418 - mean_absolute_error: 0.1561 - val_loss: 0.0499 - val_mean_absolute_error: 0.1714
Epoch 338/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0418 - mean_absolute_error: 0.1566 - val_loss: 0.0556 - val_mean_absolute_error: 0.1805
Epoch 339/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0404 - mean_absolute_error: 0.1520 - val_loss: 0.0579 - val_mean_absolute_error: 0.1824
Epoch 340/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0409 - mean_absolute_error: 0.1535 - val_loss: 0.0539 - val_mean_absolute_error: 0.1761
Epoch 341/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0421 - mean_absolute_error: 0.1559 - val_loss: 0.0563 - val_mean_absolute_error: 0.1770
Epoch 342/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0404 - mean_absolute_error: 0.1504 - val_loss: 0.0517 - val_mean_abs

360/360 [==============================] - 0s 100us/sample - loss: 0.0402 - mean_absolute_error: 0.1510 - val_loss: 0.0597 - val_mean_absolute_error: 0.1839
Epoch 386/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0399 - mean_absolute_error: 0.1512 - val_loss: 0.0627 - val_mean_absolute_error: 0.1906
Epoch 387/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0413 - mean_absolute_error: 0.1550 - val_loss: 0.0532 - val_mean_absolute_error: 0.1735
Epoch 388/500
360/360 [==============================] - 0s 94us/sample - loss: 0.0392 - mean_absolute_error: 0.1499 - val_loss: 0.0509 - val_mean_absolute_error: 0.1693
Epoch 389/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0409 - mean_absolute_error: 0.1535 - val_loss: 0.0521 - val_mean_absolute_error: 0.1744
Epoch 390/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0405 - mean_absolute_error: 0.1520 - val_loss: 0.0525 - val_mean_absolute_error: 

Epoch 433/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0398 - mean_absolute_error: 0.1506 - val_loss: 0.0571 - val_mean_absolute_error: 0.1770
Epoch 434/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0385 - mean_absolute_error: 0.1482 - val_loss: 0.0515 - val_mean_absolute_error: 0.1711
Epoch 435/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0395 - mean_absolute_error: 0.1490 - val_loss: 0.0577 - val_mean_absolute_error: 0.1817
Epoch 436/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0402 - mean_absolute_error: 0.1517 - val_loss: 0.0532 - val_mean_absolute_error: 0.1743
Epoch 437/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0383 - mean_absolute_error: 0.1464 - val_loss: 0.0532 - val_mean_absolute_error: 0.1755
Epoch 438/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0396 - mean_absolute_error: 0.1510 - val_loss: 0.0520 - val_mean_abs

360/360 [==============================] - 0s 75us/sample - loss: 0.0382 - mean_absolute_error: 0.1469 - val_loss: 0.0537 - val_mean_absolute_error: 0.1781
Epoch 482/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0391 - mean_absolute_error: 0.1490 - val_loss: 0.0556 - val_mean_absolute_error: 0.1731
Epoch 483/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0379 - mean_absolute_error: 0.1448 - val_loss: 0.0543 - val_mean_absolute_error: 0.1771
Epoch 484/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0391 - mean_absolute_error: 0.1502 - val_loss: 0.0527 - val_mean_absolute_error: 0.1763
Epoch 485/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0392 - mean_absolute_error: 0.1505 - val_loss: 0.0506 - val_mean_absolute_error: 0.1709
Epoch 486/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0382 - mean_absolute_error: 0.1453 - val_loss: 0.0495 - val_mean_absolute_error: 0

In [24]:
[loss, mae] = model_norm_minmax2.evaluate(test_features_minmax, test_labels_minmax)
print("Testing set Mean Abs Error: {}".format(mae))

92/92 [==============================] - 0s 54us/sample - loss: 0.0465 - mean_absolute_error: 0.1615
Testing set Mean Abs Error: 0.16147708892822266


In [27]:
sample1 = np.zeros_like(features[:1])
sample1[:1,:,:2] = (features[:1,:,:2] - minimum_features) / (maximum_features - minimum_features)
sample1[:1,:,2:] = features[:1,:,2:]
model_norm_minmax2.predict(sample1) * (maximum_labels - minimum_labels) + minimum_labels

array([[ 2.15819615e-01, -1.81362152e-01,  2.10094362e+00,
         3.42987204e+00,  4.09657240e+01,  4.30192828e+01,
         1.67903978e+02,  3.01728964e+02,  3.06935859e+02]])

In [28]:
labels[0]

array([  0.5,   0.3,   2.1,   2.9,  36. ,   4. , 210. , 220. , 220. ])

In [29]:
sample1 = np.zeros_like(features[:1])
sample1[:1,:,:2] = (features[:1,:,:2] - minimum_features) / (maximum_features - minimum_features)
sample1[:1,:,2:] = features[:1,:,2:]
model_norm_minmax.predict(sample1) * (maximum_labels - minimum_labels) + minimum_labels

array([[ 3.14308560e-01, -3.00958633e-01,  2.36795333e+00,
         3.09285426e+00,  5.47510862e+01,  4.78829384e+01,
         1.75575435e+02,  4.09558368e+02,  4.06648755e+02]])

In [54]:
def singleVarModel(x, y, hidden_neurons=32):
    # one variable only
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(162,6)),
        keras.layers.Dense(hidden_neurons, activation=tf.nn.relu),
        keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])
    model.compile(loss='mse', 
                              optimizer=tf.train.RMSPropOptimizer(0.001), 
                              metrics=['mae'])
    model.fit(x, y, epochs=500, validation_split=0.2)
    return model

In [50]:
reactionTimeModel = singleVarModel(train_features_minmax, train_labels_minmax[:,0])
[loss, mae] = reactionTimeModel.evaluate(test_features_minmax, test_labels_minmax[:,0])
print("Testing set Mean Abs Error: {}".format(mae))

Train on 360 samples, validate on 90 samples
Epoch 1/500
360/360 [==============================] - 0s 531us/sample - loss: 0.0869 - mean_absolute_error: 0.2581 - val_loss: 0.0852 - val_mean_absolute_error: 0.2587
Epoch 2/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0849 - mean_absolute_error: 0.2550 - val_loss: 0.0828 - val_mean_absolute_error: 0.2550
Epoch 3/500
360/360 [==============================] - 0s 100us/sample - loss: 0.0818 - mean_absolute_error: 0.2495 - val_loss: 0.0781 - val_mean_absolute_error: 0.2472
Epoch 4/500
360/360 [==============================] - 0s 100us/sample - loss: 0.0770 - mean_absolute_error: 0.2406 - val_loss: 0.0826 - val_mean_absolute_error: 0.2560
Epoch 5/500
360/360 [==============================] - 0s 103us/sample - loss: 0.0727 - mean_absolute_error: 0.2314 - val_loss: 0.0637 - val_mean_absolute_error: 0.2189
Epoch 6/500
360/360 [==============================] - 0s 106us/sample - loss: 0.0626 - mean_absolute_error: 0.

360/360 [==============================] - 0s 81us/sample - loss: 0.0329 - mean_absolute_error: 0.1468 - val_loss: 0.0305 - val_mean_absolute_error: 0.1355
Epoch 50/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0340 - mean_absolute_error: 0.1445 - val_loss: 0.0552 - val_mean_absolute_error: 0.1957
Epoch 51/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0300 - mean_absolute_error: 0.1385 - val_loss: 0.0415 - val_mean_absolute_error: 0.1572
Epoch 52/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0325 - mean_absolute_error: 0.1423 - val_loss: 0.1101 - val_mean_absolute_error: 0.2818
Epoch 53/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0316 - mean_absolute_error: 0.1433 - val_loss: 0.0289 - val_mean_absolute_error: 0.1327
Epoch 54/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0394 - mean_absolute_error: 0.1591 - val_loss: 0.0434 - val_mean_absolute_error: 0.1603

360/360 [==============================] - 0s 119us/sample - loss: 0.0206 - mean_absolute_error: 0.1181 - val_loss: 0.0295 - val_mean_absolute_error: 0.1359
Epoch 98/500
360/360 [==============================] - 0s 103us/sample - loss: 0.0260 - mean_absolute_error: 0.1327 - val_loss: 0.0249 - val_mean_absolute_error: 0.1302
Epoch 99/500
360/360 [==============================] - 0s 111us/sample - loss: 0.0271 - mean_absolute_error: 0.1312 - val_loss: 0.0302 - val_mean_absolute_error: 0.1383
Epoch 100/500
360/360 [==============================] - 0s 100us/sample - loss: 0.0285 - mean_absolute_error: 0.1362 - val_loss: 0.0245 - val_mean_absolute_error: 0.1261
Epoch 101/500
360/360 [==============================] - 0s 97us/sample - loss: 0.0193 - mean_absolute_error: 0.1141 - val_loss: 0.0675 - val_mean_absolute_error: 0.2111
Epoch 102/500
360/360 [==============================] - 0s 114us/sample - loss: 0.0312 - mean_absolute_error: 0.1412 - val_loss: 0.0408 - val_mean_absolute_error

360/360 [==============================] - 0s 67us/sample - loss: 0.0232 - mean_absolute_error: 0.1235 - val_loss: 0.0187 - val_mean_absolute_error: 0.1135
Epoch 146/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0207 - mean_absolute_error: 0.1179 - val_loss: 0.0530 - val_mean_absolute_error: 0.1872
Epoch 147/500
360/360 [==============================] - 0s 80us/sample - loss: 0.0185 - mean_absolute_error: 0.1090 - val_loss: 0.0202 - val_mean_absolute_error: 0.1139
Epoch 148/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0235 - mean_absolute_error: 0.1247 - val_loss: 0.0195 - val_mean_absolute_error: 0.1143
Epoch 149/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0219 - mean_absolute_error: 0.1232 - val_loss: 0.0314 - val_mean_absolute_error: 0.1391
Epoch 150/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0208 - mean_absolute_error: 0.1175 - val_loss: 0.0352 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 72us/sample - loss: 0.0223 - mean_absolute_error: 0.1218 - val_loss: 0.0259 - val_mean_absolute_error: 0.1270
Epoch 194/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0146 - mean_absolute_error: 0.1005 - val_loss: 0.0177 - val_mean_absolute_error: 0.1125
Epoch 195/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0225 - mean_absolute_error: 0.1221 - val_loss: 0.0239 - val_mean_absolute_error: 0.1238
Epoch 196/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0190 - mean_absolute_error: 0.1120 - val_loss: 0.0314 - val_mean_absolute_error: 0.1435
Epoch 197/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0176 - mean_absolute_error: 0.1100 - val_loss: 0.0203 - val_mean_absolute_error: 0.1175
Epoch 198/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0185 - mean_absolute_error: 0.1085 - val_loss: 0.0169 - val_mean_absolute_error: 0

...

360/360 [==============================] - 0s 67us/sample - loss: 0.0143 - mean_absolute_error: 0.0970 - val_loss: 0.0163 - val_mean_absolute_error: 0.1052
Epoch 290/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0174 - mean_absolute_error: 0.1058 - val_loss: 0.0312 - val_mean_absolute_error: 0.1439
Epoch 291/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0162 - mean_absolute_error: 0.1049 - val_loss: 0.0207 - val_mean_absolute_error: 0.1182
Epoch 292/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0177 - mean_absolute_error: 0.1091 - val_loss: 0.0188 - val_mean_absolute_error: 0.1095
Epoch 293/500
360/360 [==============================] - 0s 64us/sample - loss: 0.0162 - mean_absolute_error: 0.1036 - val_loss: 0.0146 - val_mean_absolute_error: 0.1034
Epoch 294/500
360/360 [==============================] - 0s 108us/sample - loss: 0.0168 - mean_absolute_error: 0.1059 - val_loss: 0.0243 - val_mean_absolute_error: 

360/360 [==============================] - 0s 69us/sample - loss: 0.0182 - mean_absolute_error: 0.1106 - val_loss: 0.0165 - val_mean_absolute_error: 0.1098
Epoch 338/500
360/360 [==============================] - 0s 92us/sample - loss: 0.0137 - mean_absolute_error: 0.0955 - val_loss: 0.0210 - val_mean_absolute_error: 0.1170
Epoch 339/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0154 - mean_absolute_error: 0.1001 - val_loss: 0.0164 - val_mean_absolute_error: 0.1068
Epoch 340/500
360/360 [==============================] - 0s 61us/sample - loss: 0.0103 - mean_absolute_error: 0.0850 - val_loss: 0.0220 - val_mean_absolute_error: 0.1140
Epoch 341/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0162 - mean_absolute_error: 0.1024 - val_loss: 0.0246 - val_mean_absolute_error: 0.1299
Epoch 342/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0141 - mean_absolute_error: 0.0969 - val_loss: 0.0560 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 83us/sample - loss: 0.0154 - mean_absolute_error: 0.0974 - val_loss: 0.0137 - val_mean_absolute_error: 0.1014
Epoch 386/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0124 - mean_absolute_error: 0.0887 - val_loss: 0.0177 - val_mean_absolute_error: 0.1109
Epoch 387/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0153 - mean_absolute_error: 0.1004 - val_loss: 0.0178 - val_mean_absolute_error: 0.1100
Epoch 388/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0118 - mean_absolute_error: 0.0882 - val_loss: 0.0225 - val_mean_absolute_error: 0.1216
Epoch 389/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0131 - mean_absolute_error: 0.0922 - val_loss: 0.0182 - val_mean_absolute_error: 0.1116
Epoch 390/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0153 - mean_absolute_error: 0.1014 - val_loss: 0.0205 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 75us/sample - loss: 0.0116 - mean_absolute_error: 0.0880 - val_loss: 0.0131 - val_mean_absolute_error: 0.0986
Epoch 434/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0134 - mean_absolute_error: 0.0928 - val_loss: 0.0283 - val_mean_absolute_error: 0.1336
Epoch 435/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0130 - mean_absolute_error: 0.0945 - val_loss: 0.0140 - val_mean_absolute_error: 0.0980
Epoch 436/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0147 - mean_absolute_error: 0.0975 - val_loss: 0.0175 - val_mean_absolute_error: 0.1063
Epoch 437/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0119 - mean_absolute_error: 0.0890 - val_loss: 0.0127 - val_mean_absolute_error: 0.0956
Epoch 438/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0141 - mean_absolute_error: 0.0973 - val_loss: 0.0144 - val_mean_absolute_error: 0

Epoch 481/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0141 - mean_absolute_error: 0.0950 - val_loss: 0.0197 - val_mean_absolute_error: 0.1116
Epoch 482/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0124 - mean_absolute_error: 0.0924 - val_loss: 0.0142 - val_mean_absolute_error: 0.0989
Epoch 483/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0115 - mean_absolute_error: 0.0861 - val_loss: 0.0203 - val_mean_absolute_error: 0.1141
Epoch 484/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0139 - mean_absolute_error: 0.0941 - val_loss: 0.0177 - val_mean_absolute_error: 0.1083
Epoch 485/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0100 - mean_absolute_error: 0.0812 - val_loss: 0.0452 - val_mean_absolute_error: 0.1753
Epoch 486/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0141 - mean_absolute_error: 0.0963 - val_loss: 0.0132 - val_mean_abs

In [51]:
sample1 = np.zeros_like(features[:1])
sample1[:1,:,:2] = (features[:1,:,:2] - minimum_features) / (maximum_features - minimum_features)
sample1[:1,:,2:] = features[:1,:,2:]
reactionTimeModel.predict(sample1) * (maximum_labels[0] - minimum_labels[0]) + minimum_labels[0]

array([[0.48176867]], dtype=float32)

In [52]:
vehAggModel = singleVarModel(train_features_minmax, train_labels_minmax[:,1])
[loss, mae] = vehAggModel.evaluate(test_features_minmax, test_labels_minmax[:,1])
print("Testing set Mean Abs Error: {}".format(mae))

Train on 360 samples, validate on 90 samples
Epoch 1/500
360/360 [==============================] - 0s 534us/sample - loss: 0.0985 - mean_absolute_error: 0.2739 - val_loss: 0.0936 - val_mean_absolute_error: 0.2677
Epoch 2/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0958 - mean_absolute_error: 0.2728 - val_loss: 0.0943 - val_mean_absolute_error: 0.2692
Epoch 3/500
360/360 [==============================] - 0s 103us/sample - loss: 0.0944 - mean_absolute_error: 0.2722 - val_loss: 0.0930 - val_mean_absolute_error: 0.2673
Epoch 4/500
360/360 [==============================] - 0s 103us/sample - loss: 0.0943 - mean_absolute_error: 0.2714 - val_loss: 0.0926 - val_mean_absolute_error: 0.2668
Epoch 5/500
360/360 [==============================] - 0s 114us/sample - loss: 0.0940 - mean_absolute_error: 0.2704 - val_loss: 0.0909 - val_mean_absolute_error: 0.2644
Epoch 6/500
360/360 [==============================] - 0s 97us/sample - loss: 0.0957 - mean_absolute_error: 0.2

Epoch 49/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0780 - mean_absolute_error: 0.2375 - val_loss: 0.0756 - val_mean_absolute_error: 0.2331
Epoch 50/500
360/360 [==============================] - 0s 92us/sample - loss: 0.0760 - mean_absolute_error: 0.2342 - val_loss: 0.0794 - val_mean_absolute_error: 0.2361
Epoch 51/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0816 - mean_absolute_error: 0.2427 - val_loss: 0.0783 - val_mean_absolute_error: 0.2360
Epoch 52/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0788 - mean_absolute_error: 0.2397 - val_loss: 0.0706 - val_mean_absolute_error: 0.2207
Epoch 53/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0809 - mean_absolute_error: 0.2419 - val_loss: 0.0685 - val_mean_absolute_error: 0.2183
Epoch 54/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0766 - mean_absolute_error: 0.2334 - val_loss: 0.0700 - val_mean_absolute_

360/360 [==============================] - 0s 89us/sample - loss: 0.0648 - mean_absolute_error: 0.2140 - val_loss: 0.0579 - val_mean_absolute_error: 0.1998
Epoch 98/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0687 - mean_absolute_error: 0.2204 - val_loss: 0.0613 - val_mean_absolute_error: 0.2049
Epoch 99/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0667 - mean_absolute_error: 0.2184 - val_loss: 0.0602 - val_mean_absolute_error: 0.2026
Epoch 100/500
360/360 [==============================] - 0s 131us/sample - loss: 0.0691 - mean_absolute_error: 0.2224 - val_loss: 0.0596 - val_mean_absolute_error: 0.2014
Epoch 101/500
360/360 [==============================] - 0s 147us/sample - loss: 0.0638 - mean_absolute_error: 0.2124 - val_loss: 0.0706 - val_mean_absolute_error: 0.2178
Epoch 102/500
360/360 [==============================] - 0s 103us/sample - loss: 0.0680 - mean_absolute_error: 0.2198 - val_loss: 0.0802 - val_mean_absolute_error: 

360/360 [==============================] - 0s 81us/sample - loss: 0.0648 - mean_absolute_error: 0.2093 - val_loss: 0.0638 - val_mean_absolute_error: 0.2021
Epoch 146/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0601 - mean_absolute_error: 0.2023 - val_loss: 0.0664 - val_mean_absolute_error: 0.2098
Epoch 147/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0629 - mean_absolute_error: 0.2072 - val_loss: 0.0554 - val_mean_absolute_error: 0.1916
Epoch 148/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0679 - mean_absolute_error: 0.2149 - val_loss: 0.0626 - val_mean_absolute_error: 0.2028
Epoch 149/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0587 - mean_absolute_error: 0.2010 - val_loss: 0.0583 - val_mean_absolute_error: 0.1963
Epoch 150/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0618 - mean_absolute_error: 0.2092 - val_loss: 0.0616 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 69us/sample - loss: 0.0606 - mean_absolute_error: 0.2002 - val_loss: 0.0567 - val_mean_absolute_error: 0.1932
Epoch 194/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0620 - mean_absolute_error: 0.2067 - val_loss: 0.0740 - val_mean_absolute_error: 0.2178
Epoch 195/500
360/360 [==============================] - 0s 92us/sample - loss: 0.0607 - mean_absolute_error: 0.2020 - val_loss: 0.0710 - val_mean_absolute_error: 0.2120
Epoch 196/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0584 - mean_absolute_error: 0.1973 - val_loss: 0.0589 - val_mean_absolute_error: 0.1939
Epoch 197/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0619 - mean_absolute_error: 0.2049 - val_loss: 0.0652 - val_mean_absolute_error: 0.2054
Epoch 198/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0583 - mean_absolute_error: 0.1985 - val_loss: 0.0576 - val_mean_absolute_error: 0

...

360/360 [==============================] - 0s 86us/sample - loss: 0.0563 - mean_absolute_error: 0.1896 - val_loss: 0.0602 - val_mean_absolute_error: 0.1945
Epoch 290/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0546 - mean_absolute_error: 0.1856 - val_loss: 0.0626 - val_mean_absolute_error: 0.1949
Epoch 291/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0543 - mean_absolute_error: 0.1872 - val_loss: 0.0560 - val_mean_absolute_error: 0.1894
Epoch 292/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0520 - mean_absolute_error: 0.1844 - val_loss: 0.0689 - val_mean_absolute_error: 0.2101
Epoch 293/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0570 - mean_absolute_error: 0.1917 - val_loss: 0.0531 - val_mean_absolute_error: 0.1823
Epoch 294/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0528 - mean_absolute_error: 0.1833 - val_loss: 0.0758 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 72us/sample - loss: 0.0506 - mean_absolute_error: 0.1792 - val_loss: 0.0600 - val_mean_absolute_error: 0.1938
Epoch 338/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0552 - mean_absolute_error: 0.1887 - val_loss: 0.0591 - val_mean_absolute_error: 0.1939
Epoch 339/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0535 - mean_absolute_error: 0.1861 - val_loss: 0.0548 - val_mean_absolute_error: 0.1833
Epoch 340/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0560 - mean_absolute_error: 0.1912 - val_loss: 0.0543 - val_mean_absolute_error: 0.1844
Epoch 341/500
360/360 [==============================] - 0s 64us/sample - loss: 0.0563 - mean_absolute_error: 0.1906 - val_loss: 0.0525 - val_mean_absolute_error: 0.1807
Epoch 342/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0530 - mean_absolute_error: 0.1810 - val_loss: 0.0539 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 67us/sample - loss: 0.0485 - mean_absolute_error: 0.1763 - val_loss: 0.0935 - val_mean_absolute_error: 0.2495
Epoch 386/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0553 - mean_absolute_error: 0.1896 - val_loss: 0.0650 - val_mean_absolute_error: 0.2034
Epoch 387/500
360/360 [==============================] - 0s 64us/sample - loss: 0.0526 - mean_absolute_error: 0.1856 - val_loss: 0.0673 - val_mean_absolute_error: 0.2079
Epoch 388/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0514 - mean_absolute_error: 0.1798 - val_loss: 0.0555 - val_mean_absolute_error: 0.1861
Epoch 389/500
360/360 [==============================] - 0s 70us/sample - loss: 0.0526 - mean_absolute_error: 0.1840 - val_loss: 0.0585 - val_mean_absolute_error: 0.1898
Epoch 390/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0544 - mean_absolute_error: 0.1874 - val_loss: 0.0550 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 69us/sample - loss: 0.0478 - mean_absolute_error: 0.1725 - val_loss: 0.0543 - val_mean_absolute_error: 0.1810
Epoch 434/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0539 - mean_absolute_error: 0.1885 - val_loss: 0.0550 - val_mean_absolute_error: 0.1853
Epoch 435/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0506 - mean_absolute_error: 0.1792 - val_loss: 0.0897 - val_mean_absolute_error: 0.2463
Epoch 436/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0519 - mean_absolute_error: 0.1812 - val_loss: 0.0726 - val_mean_absolute_error: 0.2151
Epoch 437/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0499 - mean_absolute_error: 0.1755 - val_loss: 0.0592 - val_mean_absolute_error: 0.1919
Epoch 438/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0521 - mean_absolute_error: 0.1817 - val_loss: 0.0753 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 64us/sample - loss: 0.0489 - mean_absolute_error: 0.1764 - val_loss: 0.0784 - val_mean_absolute_error: 0.2273
Epoch 481/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0497 - mean_absolute_error: 0.1782 - val_loss: 0.0571 - val_mean_absolute_error: 0.1857
Epoch 482/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0531 - mean_absolute_error: 0.1823 - val_loss: 0.0571 - val_mean_absolute_error: 0.1852
Epoch 483/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0501 - mean_absolute_error: 0.1773 - val_loss: 0.0706 - val_mean_absolute_error: 0.2115
Epoch 484/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0489 - mean_absolute_error: 0.1761 - val_loss: 0.0793 - val_mean_absolute_error: 0.2277
Epoch 485/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0507 - mean_absolute_error: 0.1792 - val_loss: 0.0568 - val_mean_absolute_error: 0

In [53]:
sample1 = np.zeros_like(features[:1])
sample1[:1,:,:2] = (features[:1,:,:2] - minimum_features) / (maximum_features - minimum_features)
sample1[:1,:,2:] = features[:1,:,2:]
vehAggModel.predict(sample1) * (maximum_labels[1] - minimum_labels[1]) + minimum_labels[1]

array([[0.13727355]], dtype=float32)

In [55]:
models = []
for i in range(9):
    models.append(singleVarModel(train_features_minmax, train_labels_minmax[:,i]))

Train on 360 samples, validate on 90 samples
Epoch 1/500
360/360 [==============================] - 0s 606us/sample - loss: 0.1047 - mean_absolute_error: 0.2855 - val_loss: 0.0994 - val_mean_absolute_error: 0.2792
Epoch 2/500
360/360 [==============================] - 0s 83us/sample - loss: 0.1027 - mean_absolute_error: 0.2830 - val_loss: 0.0986 - val_mean_absolute_error: 0.2790
Epoch 3/500
360/360 [==============================] - 0s 103us/sample - loss: 0.1005 - mean_absolute_error: 0.2800 - val_loss: 0.0969 - val_mean_absolute_error: 0.2766
Epoch 4/500
360/360 [==============================] - 0s 108us/sample - loss: 0.0978 - mean_absolute_error: 0.2756 - val_loss: 0.0944 - val_mean_absolute_error: 0.2733
Epoch 5/500
360/360 [==============================] - 0s 100us/sample - loss: 0.0942 - mean_absolute_error: 0.2701 - val_loss: 0.0897 - val_mean_absolute_error: 0.2632
Epoch 6/500
360/360 [==============================] - 0s 117us/sample - loss: 0.0888 - mean_absolute_error: 0.

360/360 [==============================] - 0s 75us/sample - loss: 0.0304 - mean_absolute_error: 0.1404 - val_loss: 0.0487 - val_mean_absolute_error: 0.1896
Epoch 50/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0353 - mean_absolute_error: 0.1497 - val_loss: 0.0465 - val_mean_absolute_error: 0.1823
Epoch 51/500
360/360 [==============================] - 0s 69us/sample - loss: 0.0368 - mean_absolute_error: 0.1527 - val_loss: 0.0371 - val_mean_absolute_error: 0.1491
Epoch 52/500
360/360 [==============================] - 0s 72us/sample - loss: 0.0288 - mean_absolute_error: 0.1353 - val_loss: 0.0364 - val_mean_absolute_error: 0.1552
Epoch 53/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0367 - mean_absolute_error: 0.1558 - val_loss: 0.0444 - val_mean_absolute_error: 0.1775
Epoch 54/500
360/360 [==============================] - 0s 67us/sample - loss: 0.0298 - mean_absolute_error: 0.1393 - val_loss: 0.0347 - val_mean_absolute_error: 0.1506

360/360 [==============================] - 0s 100us/sample - loss: 0.0257 - mean_absolute_error: 0.1292 - val_loss: 0.0228 - val_mean_absolute_error: 0.1225
Epoch 98/500
360/360 [==============================] - 0s 100us/sample - loss: 0.0244 - mean_absolute_error: 0.1265 - val_loss: 0.0274 - val_mean_absolute_error: 0.1340
Epoch 99/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0234 - mean_absolute_error: 0.1207 - val_loss: 0.0377 - val_mean_absolute_error: 0.1524
Epoch 100/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0263 - mean_absolute_error: 0.1302 - val_loss: 0.0274 - val_mean_absolute_error: 0.1322
Epoch 101/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0218 - mean_absolute_error: 0.1185 - val_loss: 0.0315 - val_mean_absolute_error: 0.1398
Epoch 102/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0280 - mean_absolute_error: 0.1348 - val_loss: 0.0413 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 72us/sample - loss: 0.0205 - mean_absolute_error: 0.1174 - val_loss: 0.0195 - val_mean_absolute_error: 0.1144
Epoch 146/500
360/360 [==============================] - 0s 92us/sample - loss: 0.0214 - mean_absolute_error: 0.1179 - val_loss: 0.0186 - val_mean_absolute_error: 0.1140
Epoch 147/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0165 - mean_absolute_error: 0.1057 - val_loss: 0.0535 - val_mean_absolute_error: 0.1894
Epoch 148/500
360/360 [==============================] - 0s 97us/sample - loss: 0.0230 - mean_absolute_error: 0.1201 - val_loss: 0.0199 - val_mean_absolute_error: 0.1172
Epoch 149/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0178 - mean_absolute_error: 0.1094 - val_loss: 0.0350 - val_mean_absolute_error: 0.1484
Epoch 150/500
360/360 [==============================] - 0s 97us/sample - loss: 0.0220 - mean_absolute_error: 0.1212 - val_loss: 0.0226 - val_mean_absolute_error: 0

360/360 [==============================] - 0s 86us/sample - loss: 0.0196 - mean_absolute_error: 0.1111 - val_loss: 0.0254 - val_mean_absolute_error: 0.1310
Epoch 194/500
360/360 [==============================] - 0s 78us/sample - loss: 0.0154 - mean_absolute_error: 0.1016 - val_loss: 0.0184 - val_mean_absolute_error: 0.1133
Epoch 195/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0214 - mean_absolute_error: 0.1164 - val_loss: 0.0172 - val_mean_absolute_error: 0.1118
Epoch 196/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0171 - mean_absolute_error: 0.1055 - val_loss: 0.0352 - val_mean_absolute_error: 0.1468
Epoch 197/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0198 - mean_absolute_error: 0.1134 - val_loss: 0.0231 - val_mean_absolute_error: 0.1239
Epoch 198/500
360/360 [==============================] - 0s 89us/sample - loss: 0.0150 - mean_absolute_error: 0.1006 - val_loss: 0.0363 - val_mean_absolute_error: 0

...

360/360 [==============================] - 0s 89us/sample - loss: 0.0075 - mean_absolute_error: 0.0652 - val_loss: 0.0744 - val_mean_absolute_error: 0.2140
Epoch 268/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0177 - mean_absolute_error: 0.0890 - val_loss: 0.0108 - val_mean_absolute_error: 0.0807
Epoch 269/500
360/360 [==============================] - 0s 75us/sample - loss: 0.0103 - mean_absolute_error: 0.0732 - val_loss: 0.0232 - val_mean_absolute_error: 0.1246
Epoch 270/500
360/360 [==============================] - 0s 133us/sample - loss: 0.0113 - mean_absolute_error: 0.0824 - val_loss: 0.0780 - val_mean_absolute_error: 0.2338
Epoch 271/500
360/360 [==============================] - 0s 125us/sample - loss: 0.0118 - mean_absolute_error: 0.0750 - val_loss: 0.0140 - val_mean_absolute_error: 0.0927
Epoch 272/500
360/360 [==============================] - 0s 94us/sample - loss: 0.0099 - mean_absolute_error: 0.0747 - val_loss: 0.0208 - val_mean_absolute_error:

360/360 [==============================] - 0s 89us/sample - loss: 0.0110 - mean_absolute_error: 0.0743 - val_loss: 0.0183 - val_mean_absolute_error: 0.0966
Epoch 316/500
360/360 [==============================] - 0s 100us/sample - loss: 0.0131 - mean_absolute_error: 0.0843 - val_loss: 0.0099 - val_mean_absolute_error: 0.0771
Epoch 317/500
360/360 [==============================] - 0s 92us/sample - loss: 0.0063 - mean_absolute_error: 0.0584 - val_loss: 0.0459 - val_mean_absolute_error: 0.1670
Epoch 318/500
360/360 [==============================] - 0s 94us/sample - loss: 0.0097 - mean_absolute_error: 0.0737 - val_loss: 0.0381 - val_mean_absolute_error: 0.1429
Epoch 319/500
360/360 [==============================] - 0s 97us/sample - loss: 0.0107 - mean_absolute_error: 0.0790 - val_loss: 0.0137 - val_mean_absolute_error: 0.0869
Epoch 320/500
360/360 [==============================] - 0s 100us/sample - loss: 0.0084 - mean_absolute_error: 0.0677 - val_loss: 0.0124 - val_mean_absolute_error:

360/360 [==============================] - 0s 83us/sample - loss: 0.0089 - mean_absolute_error: 0.0667 - val_loss: 0.0146 - val_mean_absolute_error: 0.0896
Epoch 364/500
360/360 [==============================] - 0s 83us/sample - loss: 0.0075 - mean_absolute_error: 0.0653 - val_loss: 0.0572 - val_mean_absolute_error: 0.1961
Epoch 365/500
360/360 [==============================] - 0s 100us/sample - loss: 0.0099 - mean_absolute_error: 0.0721 - val_loss: 0.0108 - val_mean_absolute_error: 0.0784
Epoch 366/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0086 - mean_absolute_error: 0.0692 - val_loss: 0.0224 - val_mean_absolute_error: 0.1153
Epoch 367/500
360/360 [==============================] - 0s 94us/sample - loss: 0.0078 - mean_absolute_error: 0.0671 - val_loss: 0.0146 - val_mean_absolute_error: 0.0873
Epoch 368/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0096 - mean_absolute_error: 0.0752 - val_loss: 0.0100 - val_mean_absolute_error: 

360/360 [==============================] - 0s 78us/sample - loss: 0.0128 - mean_absolute_error: 0.0830 - val_loss: 0.0089 - val_mean_absolute_error: 0.0723
Epoch 412/500
360/360 [==============================] - 0s 97us/sample - loss: 0.0059 - mean_absolute_error: 0.0577 - val_loss: 0.0120 - val_mean_absolute_error: 0.0856
Epoch 413/500
360/360 [==============================] - 0s 108us/sample - loss: 0.0099 - mean_absolute_error: 0.0713 - val_loss: 0.0105 - val_mean_absolute_error: 0.0739
Epoch 414/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0068 - mean_absolute_error: 0.0630 - val_loss: 0.0115 - val_mean_absolute_error: 0.0782
Epoch 415/500
360/360 [==============================] - 0s 95us/sample - loss: 0.0106 - mean_absolute_error: 0.0777 - val_loss: 0.0199 - val_mean_absolute_error: 0.1018
Epoch 416/500
360/360 [==============================] - 0s 92us/sample - loss: 0.0079 - mean_absolute_error: 0.0678 - val_loss: 0.0101 - val_mean_absolute_error: 

360/360 [==============================] - 0s 97us/sample - loss: 0.0083 - mean_absolute_error: 0.0694 - val_loss: 0.0110 - val_mean_absolute_error: 0.0795
Epoch 460/500
360/360 [==============================] - 0s 92us/sample - loss: 0.0088 - mean_absolute_error: 0.0699 - val_loss: 0.0106 - val_mean_absolute_error: 0.0753
Epoch 461/500
360/360 [==============================] - 0s 106us/sample - loss: 0.0083 - mean_absolute_error: 0.0680 - val_loss: 0.0141 - val_mean_absolute_error: 0.0885
Epoch 462/500
360/360 [==============================] - 0s 92us/sample - loss: 0.0070 - mean_absolute_error: 0.0601 - val_loss: 0.0218 - val_mean_absolute_error: 0.1214
Epoch 463/500
360/360 [==============================] - 0s 81us/sample - loss: 0.0063 - mean_absolute_error: 0.0575 - val_loss: 0.0118 - val_mean_absolute_error: 0.0825
Epoch 464/500
360/360 [==============================] - 0s 86us/sample - loss: 0.0084 - mean_absolute_error: 0.0646 - val_loss: 0.0530 - val_mean_absolute_error: 

In [71]:
sample1 = np.zeros_like(features[:1])
sample1[:1,:,:2] = (features[:1,:,:2] - minimum_features) / (maximum_features - minimum_features)
sample1[:1,:,2:] = features[:1,:,2:]
y = []
for i in range(9):
    y.append(np.asscalar(models[i].predict(sample1)))
y = np.array(y) * (maximum_labels - minimum_labels) + minimum_labels
y

array([1.72182722e-01, 7.01588392e-02, 2.32411978e+00, 3.55518413e+00,
       4.94340181e+01, 4.48915154e+01, 1.43001938e+02, 3.67203081e+02,
       3.24218237e+02])

In [59]:
labels[0]

array([  0.5,   0.3,   2.1,   2.9,  36. ,   4. , 210. , 220. , 220. ])